# Bert Inference

In [2]:
!pip install -q "sagemaker==2.198" "transformers==4.26.0" "datasets==2.9" "torch==1.13.1" "tqdm"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
awscli 1.29.63 requires botocore==1.31.63, but you have botocore 1.33.13 which is incompatible.
awscli 1.29.63 requires s3transfer<0.8.0,>=0.7.0, but you have s3transfer 0.8.2 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.16.1 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.1 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import sagemaker
import boto3
import pandas as pd
import json
import csv
from tqdm import tqdm
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner
)
from ast import literal_eval

from sagemaker.huggingface import HuggingFace, HuggingFaceModel
from sagemaker.s3 import S3Uploader, S3Downloader, s3_path_join
from datasets import load_dataset
from transformers import AutoTokenizer, get_constant_schedule, get_constant_schedule_with_warmup
import s3fs
import re
#from utils import summarize_hpo_results

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
#Set up sagemaker session
sess = sagemaker.Session()
sagemaker_session_bucket='sagemaker-studio-ai-lab-3'
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::956796298028:role/service-role/AmazonSageMaker-ExecutionRole-20211005T133138
sagemaker bucket: sagemaker-studio-ai-lab-3
sagemaker session region: us-east-1


In [5]:
huggingface_model = HuggingFaceModel(
    model_data=r"s3://sagemaker-studio-ai-lab-3/samples/datasets/tokenized_reviews_v4/bert_model/LA-bert-base-60k-v2-2023-12-08-21-58-47-098/output/model.tar.gz",
    role=role, # iam role with permissions to create an Endpoint
    transformers_version="4.26.0",
    pytorch_version="1.13.1", 
    py_version='py39'
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


BatchTransform to gather predictions from BERT
    must be in jsonl file type

In [6]:
#load all real data (only r_text, review_id, r_star column) from s3 (6.3 mil), then clean, then transform to jsonl, then upload clean version to s3 (one time use)

#loading data
dfs = []
for load_num in tqdm(range(0,70)):
    df = pd.read_json(f's3://sagemaker-studio-ai-lab-3/final-data/test_{load_num}.json') 
    dfs.append(df)
reviews = pd.concat (dfs, ignore_index = True)

#cleaning text comments by tokenizing text
REPLACE_BY_SPACE_RE = re.compile('--[!/(){}\[\]\n,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
def clean_text(text):
    """
        text: a string
        return: modified initial string
    """
    text = text.lower() 
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    text = BAD_SYMBOLS_RE.sub(' ', text) 
    return text
reviews['r_text'] = reviews['r_text'].apply(clean_text)

#convert text to jsonl format and uploads to s3
reviews_jsonl = f"reviews_text.jsonl"
with open(reviews_jsonl, "w+") as outfile:
    for value in tqdm(reviews['r_text']):
        json.dump({"inputs": value, "parameters":{"truncation":True, "padding":'max_length',  "max_length": 512}}, outfile) #maybe need 128 if other model performs better
        outfile.write('\n')
s3_file_uri = S3Uploader.upload(reviews_jsonl,"s3://sagemaker-studio-ai-lab-3/samples/datasets/inference/batch_transform/test_input") 

100%|██████████| 699028/699028 [00:14<00:00, 47407.54it/s]


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [ ]:
#start batch job wit jsonl files
batch_job = huggingface_model.transformer(
    instance_count=1,
    instance_type='ml.g4dn.xlarge', #ml.g4dn.xlarge
    output_path="s3://sagemaker-studio-ai-lab-3/samples/datasets/inference/batch_transform/test_output",
    strategy='SingleRecord')

batch_job.transform(
    data=s3_file_uri, 
    content_type='application/json',    
    split_type='Line')

INFO:sagemaker:Creating model with name: huggingface-pytorch-inference-2023-12-11-23-03-24-403
INFO:sagemaker:Creating transform job with name: huggingface-pytorch-inference-2023-12-11-23-03-25-139


...

In [40]:
# creating s3 uri for result file -> input file + .out
output_file = f"reviews_text.jsonl.out"
output_path = s3_path_join("s3://sagemaker-studio-ai-lab-3/samples/datasets/inference/batch_transform/test_output",output_file)

# download file
S3Downloader.download(output_path,'.')

batch_transform_result = []
with open(output_file) as f:
    for line in f:
        # converts jsonline array to normal array
        line = "[" + line.replace("[","").replace("]",",") + "]"
        batch_transform_result = literal_eval(line) 

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [41]:
dfs = []
for load_num in tqdm(range(0,70)):
    df = pd.read_json(f's3://sagemaker-studio-ai-lab-3/final-data/test_{load_num}.json') 
    dfs.append(df)
reviews = pd.concat (dfs, ignore_index = True)

100%|██████████| 70/70 [01:19<00:00,  1.14s/it]


In [42]:
print(len(batch_transform_result))
print(len(reviews))

699028
699028


## Uploading Submission

In [43]:
list(reviews.r_text[0:5])

["I will never rent from hertz anywhere again. I reserved ahead of time through AAA. I did this because I wanted to budget.the sales lady lied to me & said I had to choose 1 of the 3 insurance packages. I told her I budgeted for yay what I was told. She pushed me & said I had no choice. My Bill went up by $97 plus tax. She was cold, pushy & rude. \nI was almost crying & she didn't blink. I have never driven the new Ford Focus & she didn't go over anything. I was just a dollar sign to her",
 "Not good food. The name mean more to the hood then the food.  Thanks for being black owned.  That's it I supported their store and didn't like my 40.00 purchase.",
 "If a friend asked me, whats the one place I need to go to in St. Pete, my answer would be Mazzaro's market. The quality of the food, the wine selection, the availability of high quality food that you cant find anywhere else in town all make Mazzaro's a must see when in town. As a side note, I actually considered having this place cater

In [44]:
reviews = reviews.assign(bert_pred = batch_transform_result)
LA_pred_2023_BERT_v1 = reviews[['review_id', 'bert_pred']]

In [45]:
#change all 0 indexed labels into normal labels again
for i in tqdm(range(len(LA_pred_2023_BERT_v1.bert_pred))):
    LA_pred_2023_BERT_v1.bert_pred[i]['label'] = LA_pred_2023_BERT_v1.bert_pred[i]['label'][:-1] + str(int(LA_pred_2023_BERT_v1.bert_pred[i]['label'][-1:])+1)
    
#change into submission format
LA_pred_2023_BERT_v1['pred'] = [int(entry['label'].split('_')[-1]) for entry in LA_pred_2023_BERT_v1['bert_pred']]

#drop unnecessary col
LA_pred_2023_BERT_v1 = LA_pred_2023_BERT_v1.drop('bert_pred', axis=1)

100%|██████████| 699028/699028 [00:14<00:00, 48884.07it/s]
/tmp/ipykernel_18/4217106364.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LA_pred_2023_BERT_v1['pred'] = [int(entry['label'].split('_')[-1]) for entry in LA_pred_2023_BERT_v1['bert_pred']]


In [60]:
#saving csv to upload filepath
submission_csv = LA_pred_2023_BERT_v1
pred_upload_path = f"s3://sagemaker-studio-ai-lab-3/la/submission_csv"
submission_csv.to_csv(pred_upload_path, index = False)

In [57]:
#saving 
final_data_path = f"s3://sagemaker-studio-ai-lab-3/samples/datasets/bert60k_test_predictions"
submission_csv.to_csv(final_data_path)